In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
#         print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Writing mapper.py


In [2]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Writing reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [5]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hadoop fs -cat ${OUT_DIR}/part-* > Result.txt
cat Result.txt | sort -k2,2nr | head -7 | tail -1

is	126420


rm: `wordcount_result_1523285869010300': No such file or directory
18/04/09 14:57:52 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/09 14:57:52 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/09 14:57:54 INFO mapred.FileInputFormat: Total input files to process : 1
18/04/09 14:57:54 INFO mapreduce.JobSubmitter: number of splits:2
18/04/09 14:57:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1523238091714_0001
18/04/09 14:57:54 INFO impl.YarnClientImpl: Submitted application application_1523238091714_0001
18/04/09 14:57:55 INFO mapreduce.Job: The url to track the job: http://f54c3d191c00:8088/proxy/application_1523238091714_0001/
18/04/09 14:57:55 INFO mapreduce.Job: Running job: job_1523238091714_0001
18/04/09 14:58:01 INFO mapreduce.Job: Job job_1523238091714_0001 running in uber mode : false
18/04/09 14:58:01 INFO mapreduce.Job:  map 0% reduce 0%
18/04/09 14:58:17 INFO mapreduce.Job:  map 46% reduce 0%
18/04/09 14:5